# DataSet-Creation

In [9]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [10]:
r=requests.get('https://en.wikipedia.org/wiki/Lists_of_Telugu-language_films')
webpage=bs(r.content)
Year_2000=webpage.select("h2+ul>li>a")
l=['https://en.wikipedia.org'+i['href'] for i in Year_2000 if i.find_all(string=re.compile('20'))]
l

['https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2000',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2001',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2002',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2003',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2004',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2005',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2006',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2007',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2008',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2009',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2010',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2011',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2012',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2013',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of_2014',
 'https://en.wikipedia.org/wiki/List_of_Telugu_films_of

In [11]:
movies=requests.get(l[0])
movies_2000=bs(movies.content)
movies_2000
Annayya=movies_2000.find_all('a',string=re.compile('Annayya'))[0]['href']
Annayya

'/wiki/Annayya_(2000_film)'

In [12]:
Annayya_page_url='https://en.wikipedia.org'+Annayya
Annayya_page_req=requests.get(Annayya_page_url)
Annayya_page=bs(Annayya_page_req.content)
Info=Annayya_page.find(class_='infobox vevent')
Info
table_rows=Info.find_all("tr")
d={}
for index,row in enumerate(table_rows):
    s=''
    if index==0:
        d['Title']=row.find("th").get_text()
    elif index==1:
        continue
    elif row.select("td div ul"):
        key=row.find("th").get_text()
        k=row.select("td div ul li")
        for i in k:
            s=s+i.get_text()+","
            if '\xa0' in s:
                s=s.replace('\xa0',' ')
        d[key]=s
        
    else:
        key=row.find("th").get_text()
        value=row.find("td").get_text()
        d[key]=value
d

{'Title': 'Annayya',
 'Directed by': 'Muthyala Subbaiah',
 'Screenplay by': 'Muthyala SubbaiahBhupathi RajaSatyanand (dialogues)',
 'Story by': 'Bhupathi Raja',
 'Produced by': 'K. Venkateswara Rao',
 'Starring': 'Chiranjeevi,Soundarya,Venkat,Ravi Teja,Chandni,Sishwa,Priya,Kota Srinivasa Rao,',
 'Cinematography': 'Chota K. Naidu',
 'Edited by': 'A. Sreekar Prasad',
 'Music by': 'Mani Sharma',
 'Productioncompany': 'Sri Sai Ram Arts',
 'Distributed by': 'Geetha Arts',
 'Release date': '7 January 2000 (2000-01-07),',
 'Running time': '157 mins',
 'Country': 'India',
 'Language': 'Telugu'}

# Get All movies Info boxes

In [13]:
movies=requests.get(l[22])
movies_2022=bs(movies.content)
movies_2022



<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of Telugu films of 2022 - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"0dfcfd8a-aa0a-4ffd-bcbf-247546461b1d","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_Telugu_films_of_2022","wgTitle":"List of Telugu films of 2022","wgCurRevisionId":1078684746,"wgRevisionId":1078684746,"wgArticleId":66320357,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: url-status","CS1 Telugu-language sources (te)","Articles with short description","Short descript

In [107]:
links=[]
a=[]
#table=movies_2000.select('table.wikitable')
table=movies_2022.find_all('table',attrs={'class':'wikitable'})
#a=table[0].select("td i a")
for i in range(len(table)):
    a.extend(table[i].select("td i a"))
for i in a:
    links.append(i['href'])
links
original_links=[]
for i in links:
    original_links.append('https://en.wikipedia.org'+i)
original_links
res = []
[res.append(x) for x in original_links if x not in res]
len(res)

28

In [104]:
page_req=[]
k=[]
movies=[]
def clean_tags(soup):
    for tag in soup.find_all(["sup"]):
        tag.decompose()

for i in res:
    d={}
    page_req=requests.get(i)
    page=bs(page_req.content)
    try:
        table_rows=page.find(class_='infobox vevent').find_all("tr")
        clean_tags(page)
        for index,row in enumerate(table_rows):
            s=''
            if row.find("br"):
                    row.find("br").replace_with(",")
            if index==0:
                d['Title']=row.find("th").get_text()
            elif index==1:
                continue
            elif row.select("td div ul"):
                key=row.find("th").get_text()
                k=row.select("td div ul li")
                p=row.select("b+a")
                for i in p:
                    s=s+i.get_text()+" "
                for i in k:
                    s=s+i.get_text()+","
                    if '\xa0' in s:
                        s=s.replace('\xa0',' ')
                    if 'br' in s:
                        s=s.replace('br',',')
                d[key]=s[0:len(s)-1]
            elif row.select("th div"):
                key=row.select("th div")[0].get_text()
                if row.find("br"):
                    row.find("br").replace_with(",")
                if row.select("td sup"):
                    value=row.find("td").get_text()[0:row.find("td").get_text().index('[')]
                else:
                    value=row.find("td").get_text()
                d[key]=value
        
            else:
                header=row.find("th")
                if header:
                    key=row.find("th").get_text()
                    if row.find("br"):
                        row.find("br").replace_with(",")
                    if row.select("td sup"):
                        value=row.find("td").get_text()[0:row.find("td").get_text().index('[')]
                    else:
                        value=row.find("td").get_text()
                    d[key]=value
            if d not in movies:
                movies.append(d)
            
    except:
        continue


In [106]:
len(movies)

28

### Saving Data

In [108]:
import json

def save_data(title,data):
    with open(title,'w',encoding='utf-8') as f:
        json.dump(data,f,ensure_ascii=False,indent=2)

In [109]:
import json

def load_data(title):
    with open(title,encoding='utf-8') as f:
        return json.load(f)

In [110]:
save_data("Movies_2020_Telugu_cleaned.json",movies)

# Cleaning Data

In [111]:
movies_list=load_data('Movies_2020_Telugu_cleaned.json')

In [112]:
movies_list

[{'Title': 'Radhe Shyam',
  'Directed by': 'Radha Krishna Kumar',
  'Written by': 'Radha Krishna Kumar',
  'Produced by': 'Bhushan Kumar (Hindi),Vamsi,Pramod,Praseedha (Telugu)',
  'Starring': 'Prabhas,Pooja Hegde',
  'Cinematography': 'Manoj Paramahamsa',
  'Edited by': 'Kotagiri Venkateswara Rao',
  'Music by': 'S. Thaman Hindi version:MithoonAmaal MallikManan Bhardwaj',
  'Production,companies': 'T-Series Films,UV Creations',
  'Distributed by': 'AA Films (Hindi), Red Giant Movies (Tamil) , RD Illuminations (Malayalam)',
  'Release date': '11 March 2022 (2022-03-11)',
  'Running time': '138 minutes',
  'Languages': 'Telugu,Hindi',
  'Budget': '₹300–350 crore',
  'Box office': 'est. ₹212.76 crore (10 days)'},
 {'Title': 'Bangarraju',
  'Directed by': 'Kalyan Krishna Kurasala',
  'Screenplay by': 'Satyanand',
  'Story by': 'Kalyan Krishna Kurasala',
  'Produced by': 'Akkineni Nagarjuna',
  'Starring': 'Akkineni Nagarjuna,Ramya Krishna,Naga Chaitanya,Krithi Shetty',
  'Cinematography':

### SubTasks:
- Remove [1] refernces
- Convert running time to integer
- Convert box office and budget to integers
- Convert release date to date time object
- Split up long strings

In [ ]:
# Remove [1] refernces
# Added the line of code in info box value=row.find("td").get_text()[0:row.find("td").get_text().index('[')] and done

In [ ]:
# Split up long strings
# if row.find("br"):row.find("br").replace_with(",") to split long strings and done

In [113]:
# Convert running time to integer
int(movies_list[0]['Running time'].split(" ")[0])

138

In [114]:
for i in movies_list:
    try:
        i['Running time (min)']=int(i['Running time'].split(" ")[0])
    except:
        i['Running time (min)']=None
        

In [115]:
[i['Running time (min)'] for i in movies_list]

[138,
 160,
 124,
 154,
 145,
 None,
 122,
 None,
 113,
 132,
 118,
 None,
 126,
 86,
 145,
 None,
 141,
 None,
 None,
 187,
 None,
 152,
 None,
 None,
 None,
 None,
 None,
 None]

In [124]:
print([movie.get('Budget','N/A') for movie in movies])

['₹300–350 crore', '₹25 crore', '₹8 crore', '₹60 crore', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '₹70−75 crore', 'N/A', 'N/A', 'N/A', 'N/A', '₹550 crore', 'N/A', '₹35 crores', 'N/A', '₹60 crore', 'N/A', '₹100 crore', 'N/A', 'N/A']


In [163]:
for i in movies_list:
    try:
        k=i['Budget'].split(' ')[0][1:].split('–')
        if len(k)==2:
            i['Budget (crores)']=float((float(k[0])+float(k[1]))/2)
        else:
            i['Budget (crores)']=float(i['Budget'].split(' ')[0][1:])
    except:
        i['Budget (crores)']=None

In [164]:
for i in movies_list:
    print(i.get('Budget','N/A'),i['Budget (crores)'])

₹300–350 crore 325.0
₹25 crore 25.0
₹8 crore 8.0
₹60 crore 60.0
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
₹70−75 crore None
N/A None
N/A None
N/A None
N/A None
₹550 crore 550.0
N/A None
₹35 crores 35.0
N/A None
₹60 crore 60.0
N/A None
₹100 crore 100.0
N/A None
N/A None


In [166]:
print([movie.get('Box office','N/A') for movie in movies_list])

['est. ₹212.76 crore (10 days)', 'est. ₹63.87 crore', 'est. ₹29.7 crore', 'est. ₹20.80 crore (1 week) ', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'est. ₹192.37 crore', 'N/A', '13 crores ( gross) https://www.tollywood.net/aadavallu-meeku-johaarlu-7-days-worldwide-box-office-collections/', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']


In [173]:
for i in movies_list:
    try:
        if i['Box office'].startswith('e'):
            i['Box office (Crores)']=float(i['Box office'].split(" ")[1][1:])
        else:
            i['Box office (Crores)']=float(i['Box office'].split(" ")[0])
    except:
        i['Box office (Crores)']=None

In [174]:
for i in movies_list:
    print(i.get('Box office','N/A'),i['Box office (Crores)'])

est. ₹212.76 crore (10 days) 212.76
est. ₹63.87 crore 63.87
est. ₹29.7 crore 29.7
est. ₹20.80 crore (1 week)  20.8
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
est. ₹192.37 crore 192.37
N/A None
13 crores ( gross) https://www.tollywood.net/aadavallu-meeku-johaarlu-7-days-worldwide-box-office-collections/ 13.0
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None
N/A None


In [175]:
movies_list

[{'Title': 'Radhe Shyam',
  'Directed by': 'Radha Krishna Kumar',
  'Written by': 'Radha Krishna Kumar',
  'Produced by': 'Bhushan Kumar (Hindi),Vamsi,Pramod,Praseedha (Telugu)',
  'Starring': 'Prabhas,Pooja Hegde',
  'Cinematography': 'Manoj Paramahamsa',
  'Edited by': 'Kotagiri Venkateswara Rao',
  'Music by': 'S. Thaman Hindi version:MithoonAmaal MallikManan Bhardwaj',
  'Production,companies': 'T-Series Films,UV Creations',
  'Distributed by': 'AA Films (Hindi), Red Giant Movies (Tamil) , RD Illuminations (Malayalam)',
  'Release date': '11 March 2022 (2022-03-11)',
  'Running time': '138 minutes',
  'Languages': 'Telugu,Hindi',
  'Budget': '₹300–350 crore',
  'Box office': 'est. ₹212.76 crore (10 days)',
  'Running time (min)': 138,
  'Budget (crores)': 325.0,
  'Box office (Crores)': 212.76},
 {'Title': 'Bangarraju',
  'Directed by': 'Kalyan Krishna Kurasala',
  'Screenplay by': 'Satyanand',
  'Story by': 'Kalyan Krishna Kurasala',
  'Produced by': 'Akkineni Nagarjuna',
  'Starr

In [200]:
#Date time Format
for i in movies_list:
    try:
        i['Release date']=i['Release date'].replace(",","br")
    except:
        pass
        
    

In [201]:
for i in movies_list:
    print(i.get('Release date','N/A').split("(")[0])

11 March 2022 
14 January 2022 
12 February 2022 
11 February 2022 
14 January 2022 
1 January 2022 
7 January 2022 
14 January 2022 
14 January 2022 
15 January 2022 
28 January 2022 
11 February 2022
11 February 2022 
18 February 2022 
25 February 2022 
4 March 2022 
4 March 2022 
18 March 2022 
17 June 2022 
25 March 2022 
1 April 2022 
8 April 2022 
N/A
12 May 2022 
20 May 2022 
27 May 2022 
10 June 2022 
25 August 2022 


In [202]:
from datetime import datetime
dates=[i.get('Release date','N/A') for i in movies_list]
def clean_date(date):
    return date.split("(")[0].strip()
def date_conversion(date):
    if isinstance(date,list):
        date=date[0]
    if date=='N/A':
        return None
    date_str=clean_date(date)
    fmt="%d %B %Y"
    return datetime.strptime(date_str,fmt)


In [203]:
for i in movies_list:
    i['Release date (datetime)']=date_conversion(i.get('Release date','N/A'))

In [204]:
movies_list

[{'Title': 'Radhe Shyam',
  'Directed by': 'Radha Krishna Kumar',
  'Written by': 'Radha Krishna Kumar',
  'Produced by': 'Bhushan Kumar (Hindi),Vamsi,Pramod,Praseedha (Telugu)',
  'Starring': 'Prabhas,Pooja Hegde',
  'Cinematography': 'Manoj Paramahamsa',
  'Edited by': 'Kotagiri Venkateswara Rao',
  'Music by': 'S. Thaman Hindi version:MithoonAmaal MallikManan Bhardwaj',
  'Production,companies': 'T-Series Films,UV Creations',
  'Distributed by': 'AA Films (Hindi), Red Giant Movies (Tamil) , RD Illuminations (Malayalam)',
  'Release date': '11 March 2022 (2022-03-11)',
  'Running time': '138 minutes',
  'Languages': 'Telugu,Hindi',
  'Budget': '₹300–350 crore',
  'Box office': 'est. ₹212.76 crore (10 days)',
  'Running time (min)': 138,
  'Budget (crores)': 325.0,
  'Box office (Crores)': 212.76,
  'Release date (datetime)': datetime.datetime(2022, 3, 11, 0, 0)},
 {'Title': 'Bangarraju',
  'Directed by': 'Kalyan Krishna Kurasala',
  'Screenplay by': 'Satyanand',
  'Story by': 'Kalyan

In [206]:
save_data("Movies_2020_Telugu_cleaned_more.json",movies_list)

TypeError: Object of type datetime is not JSON serializable

In [5]:
import pickle
def save_data_pickle(name,data):
    with open(name,"wb") as f:
        pickle.dump(data,f)

In [6]:
def load_data_pickle(name):
    with open(name,"rb") as f:
        return pickle.load(f)

In [217]:
save_data_pickle("Movies_2020_Telugu_cleaned_more.pickle",movies_list)

In [218]:
a=load_data_pickle("Movies_2020_Telugu_cleaned_more.pickle")

In [219]:
a

[{'Title': 'Radhe Shyam',
  'Directed by': 'Radha Krishna Kumar',
  'Written by': 'Radha Krishna Kumar',
  'Produced by': 'Bhushan Kumar (Hindi),Vamsi,Pramod,Praseedha (Telugu)',
  'Starring': 'Prabhas,Pooja Hegde',
  'Cinematography': 'Manoj Paramahamsa',
  'Edited by': 'Kotagiri Venkateswara Rao',
  'Music by': 'S. Thaman Hindi version:MithoonAmaal MallikManan Bhardwaj',
  'Production,companies': 'T-Series Films,UV Creations',
  'Distributed by': 'AA Films (Hindi), Red Giant Movies (Tamil) , RD Illuminations (Malayalam)',
  'Release date': '11 March 2022 (2022-03-11)',
  'Running time': '138 minutes',
  'Languages': 'Telugu,Hindi',
  'Budget': '₹300–350 crore',
  'Box office': 'est. ₹212.76 crore (10 days)',
  'Running time (min)': 138,
  'Budget (crores)': 325.0,
  'Box office (Crores)': 212.76,
  'Release date (datetime)': datetime.datetime(2022, 3, 11, 0, 0)},
 {'Title': 'Bangarraju',
  'Directed by': 'Kalyan Krishna Kurasala',
  'Screenplay by': 'Satyanand',
  'Story by': 'Kalyan

# IMDB/Rotten Tomatoes/MetaScore score

In [220]:
movies=load_data_pickle("Movies_2020_Telugu_cleaned_more.pickle")

In [224]:
movies[1]

{'Title': 'Bangarraju',
 'Directed by': 'Kalyan Krishna Kurasala',
 'Screenplay by': 'Satyanand',
 'Story by': 'Kalyan Krishna Kurasala',
 'Produced by': 'Akkineni Nagarjuna',
 'Starring': 'Akkineni Nagarjuna,Ramya Krishna,Naga Chaitanya,Krithi Shetty',
 'Cinematography': 'J Yuvaraj',
 'Edited by': 'Vijay Vardhan K',
 'Music by': 'Anup Rubens',
 'Production,companies': 'Annapurna Studios,Zee Studios',
 'Release date': '14 January 2022 (2022-01-14)',
 'Running time': '160 minutes',
 'Country': 'India',
 'Language': 'Telugu',
 'Budget': '₹25 crore',
 'Box office': 'est. ₹63.87 crore',
 'Running time (min)': 160,
 'Budget (crores)': 25.0,
 'Box office (Crores)': 63.87,
 'Release date (datetime)': datetime.datetime(2022, 1, 14, 0, 0)}

In [223]:
# http://www.omdbapi.com/?apikey=[yourkey]&
# API We use to get IMDB Scores
#http://www.omdbapi.com/?i=tt3896198&apikey=f07116f8

In [276]:
import requests
import os
import urllib
import json
def get_omdb_info(title):
    base_url='http://www.omdbapi.com/?'
    parameters={'apikey': os.environ['OMDB_API_KEY'],'t':title}
    params_encoded=urllib.parse.urlencode(parameters)
    full_url=base_url+params_encoded
    return requests.get(full_url).json()

def get_rotten_tomato_score(omdb):
    ratings=omdb.get('Ratings',[])
    for i in ratings:
        if i['Source']=='Rotten Tomatoes':
            return i['Value']



In [283]:
for i in movies_list:
    title=i['Title']
    info=get_omdb_info(title)
    try:
        i['Imdb Rating']=info['imdbRating']
    except:
        i['Imdb Rating']=None
    try:
        i['MetaScore Rating']=info['Metascore']
    except:
        i['MetaScore Rating']=None
    i['RottenTomatoes Rating']=get_rotten_tomato_score(info)

In [284]:
movies_list

[{'Title': 'Radhe Shyam',
  'Directed by': 'Radha Krishna Kumar',
  'Written by': 'Radha Krishna Kumar',
  'Produced by': 'Bhushan Kumar (Hindi),Vamsi,Pramod,Praseedha (Telugu)',
  'Starring': 'Prabhas,Pooja Hegde',
  'Cinematography': 'Manoj Paramahamsa',
  'Edited by': 'Kotagiri Venkateswara Rao',
  'Music by': 'S. Thaman Hindi version:MithoonAmaal MallikManan Bhardwaj',
  'Production,companies': 'T-Series Films,UV Creations',
  'Distributed by': 'AA Films (Hindi), Red Giant Movies (Tamil) , RD Illuminations (Malayalam)',
  'Release date': '11 March 2022 (2022-03-11)',
  'Running time': '138 minutes',
  'Languages': 'Telugu,Hindi',
  'Budget': '₹300–350 crore',
  'Box office': 'est. ₹212.76 crore (10 days)',
  'Running time (min)': 138,
  'Budget (crores)': 325.0,
  'Box office (Crores)': 212.76,
  'Release date (datetime)': datetime.datetime(2022, 3, 11, 0, 0),
  'Imdb Rating': '6.9',
  'MetaScore Rating': 'N/A',
  'RottenTomatoes Rating': None},
 {'Title': 'Bangarraju',
  'Directed

In [286]:
save_data_pickle("Movies_2020_Telugu_final.pickle",movies_list)

# Save Data into Json/CSV


In [7]:
movies_list=load_data_pickle("Movies_2020_Telugu_final.pickle")

In [288]:
movies_list_copy=[movie.copy() for movie in movies_list]

In [291]:
for i in movies_list_copy:
    current_date=i['Release date (datetime)']
    if current_date:
        i['Release date (datetime)']=current_date.strftime("%d %B %Y")
    else:
        i['Release date (datetime)']=None

In [293]:
movies_list_copy[0]

{'Title': 'Radhe Shyam',
 'Directed by': 'Radha Krishna Kumar',
 'Written by': 'Radha Krishna Kumar',
 'Produced by': 'Bhushan Kumar (Hindi),Vamsi,Pramod,Praseedha (Telugu)',
 'Starring': 'Prabhas,Pooja Hegde',
 'Cinematography': 'Manoj Paramahamsa',
 'Edited by': 'Kotagiri Venkateswara Rao',
 'Music by': 'S. Thaman Hindi version:MithoonAmaal MallikManan Bhardwaj',
 'Production,companies': 'T-Series Films,UV Creations',
 'Distributed by': 'AA Films (Hindi), Red Giant Movies (Tamil) , RD Illuminations (Malayalam)',
 'Release date': '11 March 2022 (2022-03-11)',
 'Running time': '138 minutes',
 'Languages': 'Telugu,Hindi',
 'Budget': '₹300–350 crore',
 'Box office': 'est. ₹212.76 crore (10 days)',
 'Running time (min)': 138,
 'Budget (crores)': 325.0,
 'Box office (Crores)': 212.76,
 'Release date (datetime)': '11 March 2022',
 'Imdb Rating': '6.9',
 'MetaScore Rating': 'N/A',
 'RottenTomatoes Rating': None}

In [294]:
save_data("Movies_2020_Telugu_final.json",movies_list_copy)

# Save as CSV


In [8]:
import pandas as pd

df=pd.DataFrame(movies_list)
df.head()


,Title,Directed by,Written by,Produced by,Starring,Cinematography,Edited by,Music by,"Production,companies",Distributed by,...,Imdb Rating,MetaScore Rating,RottenTomatoes Rating,Screenplay by,Story by,Country,Language,"Production,company",Based on,Dialogue by
0,Radhe Shyam,Radha Krishna Kumar,Radha Krishna Kumar,"Bhushan Kumar (Hindi),Vamsi,Pramod,Praseedha (...","Prabhas,Pooja Hegde",Manoj Paramahamsa,Kotagiri Venkateswara Rao,S. Thaman Hindi version:MithoonAmaal MallikMan...,"T-Series Films,UV Creations","AA Films (Hindi), Red Giant Movies (Tamil) , R...",...,6.9,N/A,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bangarraju,Kalyan Krishna Kurasala,NaN,Akkineni Nagarjuna,"Akkineni Nagarjuna,Ramya Krishna,Naga Chaitany...",J Yuvaraj,Vijay Vardhan K,Anup Rubens,"Annapurna Studios,Zee Studios",NaN,...,6.6,N/A,None,Satyanand,Kalyan Krishna Kurasala,India,Telugu,NaN,NaN,NaN
2,DJ Tillu,Vimal Krishna,"Vimal Krishna,Siddhu Jonnalagadda",Suryadevara Naga Vamsi,"Siddhu Jonnalagadda,Neha Shetty",Sai Prakash Ummadisingu,Naveen Nooli,"Score:,S. Thaman,Songs: Sricharan PakalaRam Mi...","Sithara Entertainments,Fortune Four Cinemas",NaN,...,7.6,N/A,None,NaN,NaN,India,Telugu,NaN,NaN,NaN
3,Khiladi,Ramesh Varma,Ramesh Varma,"Satyanarayana Koneru,Ramesh Varma","Ravi Teja,Arjun Sarja,Unni Mukundan,Meenakshi ...","Sujith Vaassudev,G.K.Vishnu",Amar Reddy Kudumula,Devi Sri Prasad,"Pen Studios,A Studios",Pen Studios,...,7.3,N/A,None,NaN,NaN,India,Telugu,NaN,NaN,NaN
4,Rowdy Boys,Sree Harsha Konuganti,Sree Harsha Konuganti,"Dil Raju,Shirish","Ashish,Anupama Parameswaran",R. Madhi,Madhu,Devi Sri Prasad,NaN,NaN,...,6.5,N/A,None,NaN,NaN,India,Telugu,Sri Venkateswara Creations,NaN,NaN


In [297]:
df.to_csv("Movies_2020_Telugu_final.csv")

In [306]:
df.sort_values('Imdb Rating',ascending=False)[df['Imdb Rating']!='N/A']

C:\Users\ruthv\AppData\Local\Temp/ipykernel_3060/71280529.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.sort_values('Imdb Rating',ascending=False)[df['Imdb Rating']!='N/A']


,Title,Directed by,Written by,Produced by,Starring,Cinematography,Edited by,Music by,"Production,companies",Distributed by,...,Imdb Rating,MetaScore Rating,RottenTomatoes Rating,Screenplay by,Story by,Country,Language,"Production,company",Based on,Dialogue by
7,Super Machi,Puli Vasu,Puli Vasu,Rizwan,"Kalyaan Dhev,Rachita Ram",Shyam K. Naidu,Marthand K. Venkatesh,S. Thaman,NaN,NaN,...,9.2,N/A,None,NaN,NaN,India,Telugu,Rizwan Entertainment,NaN,NaN
19,RRR,S. S. Rajamouli,NaN,D. V. V. Danayya,"N. T. Rama Rao Jr.,Ram Charan",K. K. Senthil Kumar,A. Sreekar Prasad,M. M. Keeravani,NaN,"Pen Studios (North India),Lyca Productions (Ta...",...,9.0,N/A,None,S. S. Rajamouli,K. V. Vijayendra Prasad\n,India,Telugu,DVV Entertainments,NaN,Sai Madhav Burra
5,Aasha Encounter,NaN,Anand Chandra,Anuraag Kancharla,"Srikant Iyengar, Sonia Akula, Praveen raj",Jagadeesh Cheekati & Josh,Srikanth patnaik & Manish Thakur,Anand,NaN,NaN,...,8.9,N/A,None,NaN,NaN,India,Telugu,Anuraag Kancharla Productions,NaN,NaN
13,Son of India,Diamond Ratna Babu,Diamond Ratna Babu,Vishnu Manchu,"Mohan Babu,Meena,Srikanth,Pragya Jaiswal",Sarvesh Murari,Gautham Raju,Ilaiyaraaja,"24 Frames Factory,Sree Lakshmi Prasanna Pictures",NaN,...,8.9,N/A,None,NaN,NaN,India,Telugu,NaN,NaN,NaN
8,The American Dream,NaN,Dr. Vighnesh Koushik,Dr. Pradeep Reddy,"Prince Cecil,Neha Krishna","Adam Chapman,Abhiraj Nair",Sasank Vupputuri,Abhinay TJ,NaN,NaN,...,8.2,N/A,None,NaN,NaN,India,Telugu,NaN,NaN,NaN
14,Bheemla Nayak,Saagar K Chandra,NaN,Suryadevara Naga Vamsi,"Pawan Kalyan,Rana Daggubati,Nithya Menen,Samyu...",Ravi K. Chandran,Naveen Nooli,S. Thaman,NaN,NaN,...,8.1,N/A,None,Trivikram Srinivas,NaN,India,Telugu,Sithara Entertainments,"Ayyappanum Koshiyum (2020),by Sachy",NaN
21,Ghani,Kiran Korrapati,Kiran Korrapati,"Sidhu Mudda,Allu Bobby","Varun Tej,Saiee Manjrekar,Jagapathi Babu,Upend...",George C. Williams,Marthand K Venkatesh,S. Thaman,"Renaissance Pictures,Allu Bobby Company",Geetha Arts,...,8.0,N/A,None,NaN,NaN,India,Telugu,NaN,NaN,NaN
9,Hero,Sriram Adittya,Sriram Adittya,Padmavathi Galla,"Ashok Galla,Niddhi Agerwal","Sameer Reddy,Richard Prasad",Prawin Pudi,Ghibran,NaN,NaN,...,7.9,85,None,NaN,NaN,India,Telugu,Amara Raja Media and Entertainment,NaN,NaN
2,DJ Tillu,Vimal Krishna,"Vimal Krishna,Siddhu Jonnalagadda",Suryadevara Naga Vamsi,"Siddhu Jonnalagadda,Neha Shetty",Sai Prakash Ummadisingu,Naveen Nooli,"Score:,S. Thaman,Songs: Sricharan PakalaRam Mi...","Sithara Entertainments,Fortune Four Cinemas",NaN,...,7.6,N/A,None,NaN,NaN,India,Telugu,NaN,NaN,NaN
10,Good Luck Sakhi,Nagesh Kukunoor,Nagesh Kukunoor,Sudheer Chandra Padiri,"Keerthy Suresh,Aadhi Pinisetty,Jagapathi Babu",Chirantan Das,Sreekar Prasad,Devi Sri Prasad,NaN,Sri Venkateswara Creations,...,7.3,N/A,None,NaN,NaN,India,Telugu,Worth A Shot Motion Arts,NaN,NaN
